In [222]:
import numpy as np
import pandas as pd

In [229]:
DB = {}
B = 20 #MHz; bandwidth of each block
M = 10 #number of f-blocks
d_max = 300 #m; max distance between two neighbors
epsilon = -1e-5
alpha = 2.5
noise = 1e-5
P = 1 #W
R = 30.0 #m
N_ITER = 100
turn_count = 100

In [230]:
class Node:
  def __init__(self, x, y, ID):
    self.x = x
    self.y = y
    self.ID = str(ID)
    self.neighbors = {}
    self.neighbor_nodes = []
    self.BSI = None
    self.datarate_min = None

  def init_BSI(self):
    self.BSI = np.ones(M, dtype='float64')

  def publish_BSI(self, DB):
    DB[self.ID] = self.BSI

  def add_neighbor(self, ID, d):
    self.neighbors[str(ID)] = d
    self.neighbor_nodes.append(ID)

  def update_BSI(self, N, DB, node_list):
    if len(self.neighbor_nodes)>0:
        # collect the BSI of neighbors and scale them by 1/distance^2
        score = np.zeros(M, dtype='float64')
        for nn in self.neighbor_nodes:
            # suggestion from the neighbours
            this_BSI = -np.power(max(1,self.neighbors[str(nn)]), -alpha)*DB[str(nn)]
            score = score + this_BSI

        # update the BSI by occupying the positive scores
        not_chosen = []

        # social learning
        for m in range(M):
            if score[m] > epsilon:
                self.BSI[m] = 1.0
            else:
                self.BSI[m] = -1.0
                not_chosen.append(m)

        # check if the datarate requirements are satisfied:
        # by calculating the performance in each f-block
        self.publish_BSI(DB)
        datarate_achieved = get_datarate(int(self.ID), N, node_list)

        # self learning
        while len(not_chosen) > 2: 
            if datarate_achieved >= self.min_datarate:
                break

            k_max = not_chosen[np.random.randint(len(not_chosen))]
            self.BSI[k_max] = 1.0
            not_chosen.remove(k_max)

            self.publish_BSI(DB)
            datarate_achieved = get_datarate(int(self.ID), N, node_list)            

    self.publish_BSI(DB)
    


In [231]:
def get_datarates(opt, N, node_list):
    datarate = np.zeros(N, dtype='float64')
    SE = np.zeros(N, dtype='float64') # average spectral efficiency
    n_iter = 1000

    for iter in range(n_iter):
        H = np.random.exponential(1, (N, N))

    for i in range(N):
        for j in range(N):
            H[i][j] = H[j][i] 
    
    for i in range(N):
        BSI_i = node_list[i].BSI
        c = 0
        for m in range(M):
            if BSI_i[m] > 0:
                I_power = 0.0
                for j in range(N):
                    if j!=i:
                        if node_list[j].BSI[m] > 0:
                            d = distance(i, j, node_list)
                            I_power += H[i][j]*P*np.power(max(1.0,d), -alpha)

                SINR = H[i][i]*P*np.power(max(1.0,R), -alpha)/(noise + I_power)
                datarate[i] += B*np.log2(1 + SINR) 
                c += 1
                SE[i] += datarate[i]/(c*B)
    if opt:
        return datarate/n_iter
    return SE/n_iter


def get_datarate(i, N, node_list):
    datarate = 0.0
    BSI_i = node_list[i].BSI
    for m in range(M):
        if BSI_i[m] > 0:
            I_power = 0.0
            for j in range(N):
                if j!=i:
                    if node_list[j].BSI[m] > 0:
                        d = distance(i, j, node_list)
                        I_power += P*np.power(max(1.0,d), -alpha)
            SINR = P*np.power(max(1.0,R), -alpha)/(noise + I_power)
            datarate += B*np.log2(1 + SINR)

    return datarate


def get_datarates_all(opt, N, node_list):
    n_iter = N_ITER
    datarate = np.zeros((N, n_iter), dtype='float64')
    SE = np.zeros((N, n_iter), dtype='float64')

    for iter in range(n_iter):
        H = np.random.exponential(1, (N, N))

        for i in range(N):
            for j in range(N):
                H[i][j] = H[j][i] 

        for i in range(N):
            BSI_i = node_list[i].BSI
            c = 0
            for m in range(M):
                if BSI_i[m] > 0:
                    I_power = 0.0
                    for j in range(N):
                        if j!=i:
                            if node_list[j].BSI[m] > 0:
                                d = distance(i, j, node_list)
                                I_power += H[i][j]*P*np.power(max(1.0,d), -alpha)
                    SINR = H[i][i]*P*np.power(max(1.0,R), -alpha)/(noise + I_power)
                    datarate[i][iter] += B*np.log2(1 + SINR)
                    c+= 1 
                SE[i][iter] += datarate[i][iter]/(c*B)

    if opt:
        return SE

    return datarate

In [232]:
def distance(i, j, node_list):
    x_i = node_list[i].x
    y_i = node_list[i].y
    x_j = node_list[j].x
    y_j = node_list[j].y
    
    return np.sqrt( np.square(x_i-x_j) + np.square(y_i-y_j) )

In [233]:
def play_grid(row, col):
    DB = {}
    node_list = []
    
    data = pd.read_csv('./data/grids/'+ str(col) +'_'+ str(row) +'.csv')
    X = list(data['X'])
    Y = list(data['Y'])
    N = len(X)

    for i in range(N):
        node_list.append(Node(X[i], Y[i], i))
        
    for i in range(N):
        for j in range(i+1, N):
            d = distance(i, j, node_list)
            
            if d < d_max:
                node_list[i].add_neighbor(j, d)
                node_list[j].add_neighbor(i, d)
                
    # init BSI of all nodes
    for i in range(N):
        node_list[i].init_BSI()
        node_list[i].publish_BSI(DB)
        
    # set min datarate requirement
    for i in range(N):
        node_list[i].min_datarate = get_datarate(i, N, node_list)
        
    # create a Trigger Poisson sequence
    poisson_triggers = list(np.random.randint(N, size=turn_count*N))
    
    # greedy results
    D_greedy = get_datarates_all(0, N, node_list)
    
    # sequential decisions
    for i in poisson_triggers:
        node_list[i].update_BSI(N, DB, node_list)
    
    # democratic results
    D_demo = get_datarates_all(0, N, node_list)
    
    np.savetxt("./data/datarates/gredy_"+str(col)+"_"+str(row)+".csv", \
               D_greedy, delimiter=",")
    np.savetxt("./data/datarates/demo_"+str(col)+"_"+str(row)+".csv", \
               D_demo, delimiter=",")
    

In [234]:
grids = 50

for i in range(grids):
    for j in range(grids):
        if (j+ 1+ grids*i)%250 == 0:
            print(str( (j+ 1+ grids*i)/25.0) )
        play_grid(i, j)

/var/folders/ds/8r4fp2095114bxbwmtj9b3y40000gn/T/ipykernel_90911/4189426427.py:77: RuntimeWarning: invalid value encountered in double_scalars
  SE[i][iter] += datarate[i][iter]/(c*B)


10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
